In [1]:
from pathlib import Path
import re

In [2]:
mice = ['bt8564', 'bt3565', 'bt0952', 'bt1617', 'bt0920', 'bt0975', 'bt1003', 'bt1009', 'bt3018','bt1010','bt3019','bt6544','bt1018','bt1021', 'bt1356', 'bt1357', 'bt1123', 'bt0962', 'bt1093']
mouse_dirs = [Path(f'/adata/electro/{mouse}/') for mouse in mice]
paths = [path for mouse_dir in mouse_dirs for path in mouse_dir.iterdir() if (path.is_dir()) and (re.search('0105$', str(path)) or re.search('0107$', str(path)) or re.search('0109$', str(path)))]
clustered_paths = [path for path in paths if Path(f"{path}/cluster_group.tsv").exists()]
sessions = [str(path).split("/")[-1] for path in paths]; clustered_sessions = [str(path).split("/")[-1] for path in clustered_paths]
print(f"Sessions:{sessions}\nNumber of sessions:{len(sessions)}\nClustered sessions:{len(clustered_sessions)}")

Sessions:['bt8564-23072021-0105', 'bt8564-26072021-0105', 'bt8564-25072021-0105', 'bt8564-24072021-0105', 'bt8564-21072021-0105', 'bt3565-20082021-0105', 'bt3565-14082021-0105', 'bt3565-21082021-0105', 'bt3565-19082021-0105', 'bt3565-16082021-0105', 'bt3565-27082021-0105', 'bt3565-22082021-0105', 'bt3565-26082021-0105', 'bt3565-15082021-0105', 'bt3565-25082021-0105', 'bt3565-24082021-0105', 'bt3565-18082021-0105', 'bt3565-23082021-0105', 'bt3565-13082021-0105', 'bt3565-17082021-0105', 'bt0952-02092021-0105', 'bt0952-05092021-0105', 'bt0952-04092021-0105', 'bt0952-06092021-0105', 'bt0952-03092021-0105', 'bt0952-01092021-0105', 'bt1617-16092021-0105', 'bt1617-24092021-0105', 'bt1617-15092021-0105', 'bt1617-10092021-0105', 'bt1617-20092021-0105', 'bt1617-21092021-0105', 'bt1617-17092021-0105', 'bt1617-14092021-0105', 'bt1617-09092021-0105', 'bt1617-23092021-0105', 'bt1617-22092021-0105', 'bt1617-08092021-0105', 'bt0920-12082021-0105', 'bt0920-11082021-0105', 'bt0920-10082021-0105', 'bt092

In [3]:
def save_waveforms(path, block_size=200, overwrite=False, n_spikes=5000):
    %load_ext autoreload
    %autoreload
    import numpy as np
    from spikeA.Session import Kilosort_session
    from spikeA.Session import Session
    from spikeA.Spike_train_loader import Spike_train_loader
    from spikeA.Cell_group import Cell_group
    from spikeA.Dat_file_reader import Dat_file_reader
    from spikeA.Intervals import Intervals
    from spikeA.Spike_waveform import Spike_waveform
    import os

    # load session
    name = str(path).split("/")[-1]
    ses = Kilosort_session(name=name,path=path)
    ses.load_parameters_from_files()
    mouse_name=name.split('-')[0]
    date_name=name.split('-')[1]
    name = str(path).split("/")[-1]
    stl = Spike_train_loader()
    stl.load_spike_train_kilosort(ses)
    cg = Cell_group(stl)
    
    number_of_trials = len(ses.desen)
    stim_trial = [i for i, j in enumerate(ses.stimulation) if j !='none']
    if stim_trial:
        recording_channels=ses.n_channels-2
    else:
        recording_channels=ses.n_channels-1
    file=f"{path}/{mouse_name}-{date_name}-010{number_of_trials}.waveforms.npy"
    print(file)
    
    if not os.path.exists(file) or overwrite==True:
        print(f'creating waveforms for {name}')
        df = Dat_file_reader(file_names=[f"{path}/{mouse_name}-{date_name}-010{number_of_trials}.dat"], n_channels=ses.n_channels)
        
        #template for array:
        array=np.zeros(recording_channels*block_size*len(cg.neuron_list))
        array=np.reshape(array, (recording_channels,block_size, len(cg.neuron_list)))
                
        for i,n in enumerate(cg.neuron_list):
            sw = Spike_waveform(session=ses, dat_file_reader=df, spike_train=n.spike_train)
            sw.mean_waveform(block_size=block_size, channels=np.asarray(range(recording_channels)), n_spikes=n_spikes) #calculate the mean waveforms for all channels over a time interval of 200 ms
            array[:,:,i]=sw.mean_waveforms

        np.save(file = file , arr = array)
    else:
        print(f"waveforms for {name} have already been created")

In [4]:
for path in clustered_paths:
    save_waveforms(path=path, overwrite=False, n_spikes=5000)

/adata/electro/bt8564/bt8564-23072021-0105/bt8564-23072021-0105.waveforms.npy
waveforms for bt8564-23072021-0105 have already been created
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/adata/electro/bt8564/bt8564-26072021-0105/bt8564-26072021-0105.waveforms.npy
waveforms for bt8564-26072021-0105 have already been created
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/adata/electro/bt8564/bt8564-25072021-0105/bt8564-25072021-0105.waveforms.npy
waveforms for bt8564-25072021-0105 have already been created
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/adata/electro/bt8564/bt8564-24072021-0105/bt8564-24072021-0105.waveforms.npy
waveforms for bt8564-24072021-0105 have already been created
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/adata/electro/bt8564/bt8564-21072021-0105/bt8564-21072021-0105.waveforms.npy
waveforms for 